# Strands Agents と AgentCore Memory（長期記憶）

## はじめに

このチュートリアルでは、フックを介して AgentCore Memory と統合された Strands エージェントを使用し、**インテリジェントなカスタマーサポートエージェント**を構築する方法を示します。顧客とのやり取り履歴、購入詳細の記憶、および以前の会話とユーザー設定に基づくパーソナライズされたサポートのために、長期記憶に焦点を当てます。

### チュートリアルの詳細

| 項目              | 詳細                                                                             |
|:------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 長期会話型                                                                       |
| エージェントタイプ  | カスタマーサポート                                                               |
| エージェントフレームワーク | Strands Agents                                                              |
| LLM モデル         | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore Semantic および User Preferences Memory 抽出、Memory の保存と取得のためのフック |
| 例の複雑さ         | 中級                                                                             |

学習内容:
- 長期戦略での AgentCore Memory のセットアップ
- 自動保存と取得のためのメモリフックの作成
- 永続的なメモリを持つ Strands エージェントの構築
- 以前のやり取りからのコンテキストを活用した顧客問題の処理

### シナリオコンテキスト
この例では、**カスタマーサポートのユースケース**を構築します。エージェントは注文履歴、設定、以前の問題を含む顧客コンテキストを記憶し、よりパーソナライズされた効果的なサポートを可能にします。顧客との会話はメモリフックを使用して自動的に保存され、重要な詳細が失われることはありません。セマンティックやユーザー設定などの複数のメモリ戦略を使用することで、エージェントは幅広い関連情報をキャプチャできます。このセットアップにより、エージェントは顧客の履歴と設定を完全に把握した上で問題を解決できます。さらに、エージェントはウェブ検索機能と統合されており、必要に応じて最新の製品情報やトラブルシューティングガイダンスを簡単に提供できます。

## アーキテクチャ

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 前提条件

このチュートリアルを実行するには以下が必要です:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 権限を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

## ステップ 1: 依存関係のインストールとセットアップ
このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義することから始めましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import json
from typing import Dict
from datetime import datetime
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("customer-support")

# Import required modules
from strands import Agent, tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from ddgs import DDGS

In [ ]:
# Configuration - Replace with the correct values
REGION = "us-west-2"
CUSTOMER_ID = "customer_001"
SESSION_ID = f"support_{datetime.now().strftime('%Y%m%d%H%M%S')}"

## ステップ 2: カスタマーサポート用のメモリリソースの作成

カスタマーサポートには、複数のメモリ戦略を使用します:
- **USER_PREFERENCE**: 顧客の設定と行動をキャプチャ
- **SEMANTIC**: 注文事実と製品情報を保存

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "CustomerSupportMemory"

# Define memory strategies for customer support
strategies = [
    {
        StrategyType.USER_PREFERENCE.value: {
            "name": "CustomerPreferences",
            "description": "Captures customer preferences and behavior",
            "namespaces": ["support/customer/{actorId}/preferences"]
        }
    },
    {
        StrategyType.SEMANTIC.value: {
            "name": "CustomerSupportSemantic",
            "description": "Stores facts from conversations",
            "namespaces": ["support/customer/{actorId}/semantic"],
        }
    }
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies,         # Define the memory strategies
        description="Memory for customer support agent",
        event_expiry_days=90,          # Memories expire after 90 days
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memoryId=memory_id,max_wait = 300)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

メモリに割り当てた戦略が含まれているか確認しましょう

In [ ]:
strategies = client.get_memory_strategies(memory_id)
print(json.dumps(strategies, indent=2, default=str))

## ステップ 3: エージェントツールの作成

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for product information, troubleshooting guides, or support articles.
    
    Args:
        query: Search query for product info or troubleshooting
        max_results: Maximum number of results to return
    
    Returns:
        Search results with titles and snippets
    """
    try:
        results = DDGS().text(query, region="us-en", max_results=max_results)
        if not results:
            return "No search results found."
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(f"{i}. {result.get('title', 'No title')}\n   {result.get('body', 'No description')}")
        
        return "\n".join(formatted_results)
    except RatelimitException:
        return "Rate limit reached: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"Search Error: {d}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

@tool
def check_order_status(order_number: str) -> str:
    """Check the status of a customer order.
    
    Args:
        order_number: The order number to check
    
    Returns:
        Order status information
    """
    # Simulate order lookup
    mock_orders = {
        "123456": "iPhone 15 Pro - Delivered on June 5, 2025",
        "654321": "Sennheiser Headphones - Delivered on June 25, 2025, 1-year warranty active",
        "789012": "Samsung Galaxy S23 - In transit, expected delivery on July 1, 2025",
    }
    
    return mock_orders.get(order_number, f"Order {order_number} not found. Please verify the order number.")

logger.info("✅ Check Order Status tool ready")

## ステップ 4: カスタマーサポート用のメモリフックプロバイダーの作成
フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。カスタムフックプロバイダーは、以下の方法でカスタマーサポートのコンテキストを自動的に管理します:
- 各応答後に**サポートのやり取りを保存**
- 新しいクエリを処理する際に、以前の注文と設定から**関連するコンテキストを取得して注入**

これにより、手動管理なしでシームレスなメモリ体験が実現されます

In [ ]:
# Helper function to get namespaces from memory strategies list
def get_namespaces(mem_client: MemoryClient, memory_id: str) -> Dict:
    """Get namespace mapping for memory strategies."""
    strategies = mem_client.get_memory_strategies(memory_id)
    return {i["type"]: i["namespaces"][0] for i in strategies}

In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    """Memory hooks for customer support agent"""
    
    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client
        self.namespaces = get_namespaces(self.client, self.memory_id)

    
    def retrieve_customer_context(self, event: MessageAddedEvent):
        """Retrieve customer context before processing support query"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                # Retrieve customer context from all namespaces
                all_context = []
                
                # Get actor_id from agent state
                actor_id = event.agent.state.get("actor_id")
                if not actor_id:
                    logger.warning("Missing actor_id in agent state")
                    return
                
                for context_type, namespace in self.namespaces.items():
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=actor_id),
                        query=user_query,
                        top_k=3
                    )
                    
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get('content', {})
                            if isinstance(content, dict):
                                text = content.get('text', '').strip()
                                if text:
                                    all_context.append(f"[{context_type.upper()}] {text}")
                
                # Inject customer context into the query
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = (
                        f"Customer Context:\n{context_text}\n\n{original_text}"
                    )
                    logger.info(f"Retrieved {len(all_context)} customer context items")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve customer context: {e}")
    
    def save_support_interaction(self, event: AfterInvocationEvent):
        """Save support interaction after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last customer query and agent response
                customer_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                if customer_query and agent_response:
                    # Get session info from agent state
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")
                    
                    if not actor_id or not session_id:
                        logger.warning("Missing actor_id or session_id in agent state")
                        return
                    
                    # Save the support interaction
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(customer_query, "USER"), (agent_response, "ASSISTANT")]
                    )
                    logger.info("Saved support interaction to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register customer support memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)
        logger.info("Customer support memory hooks registered")

### ステップ 5: カスタマーサポートエージェントの作成

In [ ]:
# Create memory hooks for customer support
support_hooks = CustomerSupportMemoryHooks(memory_id, client)

# Create customer support agent
support_agent = Agent(
    hooks=[support_hooks],
    model ="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[web_search, check_order_status],
    state={"actor_id": CUSTOMER_ID, "session_id": SESSION_ID},
    system_prompt="""You are a helpful customer support agent with access to customer history and order information. 
    
    Your role:
    - Help customers with their orders, returns, and product issues
    - Use customer context to provide personalized support
    - Search for product information when needed
    - Be empathetic and solution-focused
    - Reference previous orders and preferences when relevant
    
    Always be professional, helpful, and aim to resolve customer issues efficiently."""
)

print("✅ カスタマーサポートエージェントがメモリ機能付きで作成されました")

### ステップ 6: 顧客履歴のシード

メモリ機能をデモンストレーションするために、いくつかの以前の顧客とのやり取りを追加しましょう。

In [ ]:
# Seed with previous customer interactions
previous_interactions = [
    ("I bought a new iPhone 15 Pro on June 1st, 2025. Order number is 123456.", "USER"),
    ("Thank you for your purchase! I can see your iPhone 15 Pro order #123456 was delivered successfully. How can I help you today?", "ASSISTANT"),
    ("I also ordered Sennheiser headphones on June 20th. Order number 654321. They came with 1-year warranty.", "USER"),
    ("Perfect! I have your Sennheiser headphones order #654321 on file with the 1-year warranty. Both your iPhone and headphones should work great together.", "ASSISTANT"),
    ("I'm looking for a good laptop. I prefer ThinkPad models.", "USER"),
    ("Great choice! ThinkPads are excellent for their durability and performance. Let me help you find the right model for your needs.", "ASSISTANT")
]

# Save previous interactions
try:
    client.create_event(
        memory_id=memory_id,
        actor_id=CUSTOMER_ID,
        session_id="previous_session",
        messages=previous_interactions
    )
    print("✅ 顧客履歴をシードしました")
except Exception as e:
    print(f"⚠️ 履歴のシード中にエラー: {e}")

#### エージェントの準備完了

### カスタマーサポートシナリオのテスト

In [ ]:
# Test 1: Customer reports iPhone issue
response1 = support_agent("My iPhone is running very slow and gets hot when charging. Can you help?")
print(f"サポートエージェント: {response1}")

In [ ]:
# Test 2: Bluetooth connectivity issue
response2 = support_agent("My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?")
print(f"サポートエージェント: {response2}")

In [ ]:
# Test 3: Check order status
response3 = support_agent("Can you check the status of my recent orders?")
print(f"サポートエージェント: {response3}")

In [ ]:
# Test 4: Product recommendation based on preferences
response4 = support_agent("I'm still interested in buying a laptop. What ThinkPad models do you recommend?")
print(f"サポートエージェント: {response4}")

### 顧客メモリの保存確認

In [ ]:
# Check stored customer memories
print("\n📚 顧客メモリサマリー:")
print("=" * 50)

namespaces_dict = get_namespaces(client, memory_id)
for context_type, namespace_template in namespaces_dict.items():
    namespace = namespace_template.replace("{actorId}", CUSTOMER_ID)
    
    try:
        memories = client.retrieve_memories(
            memory_id=memory_id,
            namespace=namespace,
            query="customer orders and preferences",
            top_k=3
        )
        
        print(f"\n{context_type.upper()} ({len(memories)} 件):")
        for i, memory in enumerate(memories, 1):
            if isinstance(memory, dict):
                content = memory.get('content', {})
                if isinstance(content, dict):
                    text = content.get('text', '')[:150] + "..."
                    print(f"  {i}. {text}")
                    
    except Exception as e:
        print(f"{context_type} メモリの取得エラー: {e}")

print("\n" + "=" * 50)

#### カスタマーサポートチュートリアル完了!
主な学び:
- メモリフックはサポートセッション全体で顧客コンテキストを自動的に管理します
- マルチ戦略メモリは会話から注文、設定、事実をキャプチャします
- エージェントは顧客履歴に基づいてパーソナライズされたサポートを提供できます
- 注文検索とウェブ検索機能のためのツールを統合できます
- 永続的なメモリによりカスタマーサポートがより効率的になります

## クリーンアップ

### オプション: メモリリソースの削除

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory_and_wait(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")